In [189]:
import pandas as pd
import numpy as np
import fredapi as fa
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# 1. Enhanced Data Sourcing from FRED API with additional series
def fetch_fred_data(start_date='2000-01-01', end_date=datetime.today().strftime('%Y-%m-%d'), api_key=None):
    """Fetch Treasury yield data and additional economic indicators from FRED using the API."""
    # Get API key from environment variable if not provided
    if api_key is None:
        api_key = os.environ.get('FRED_API_KEY')
        if api_key is None:
            raise ValueError("FRED API key must be provided either directly or via FRED_API_KEY environment variable")
    
    # Initialize FRED API client
    fred = fa.Fred(api_key=api_key)
    
    # Define series to fetch
    series_dict = {
        # Original series
        'DGS3': '3_year_yield',
        'DGS5': '5_year_yield',
        'DGS7': '7_year_yield',
        'DGS10': '10_year_yield',
        'DGS20': '20_year_yield',
        'CPIAUCSL': 'cpi',
        'FEDFUNDS': 'fed_funds_rate',
        'VIXCLS': 'vix',
        
        # New series
        'DGS2': '2_year_yield',
        'CPILFESL': 'core_cpi',
        'WALCL': 'fed_assets',
        'DTWEXBGS': 'usd_index',
        'GDP': 'gdp'  # For other calculations
    }
    
    # Initialize dataframe to store results
    all_data = {}
    
    # Fetch each series
    for series_id, series_name in series_dict.items():
        print(f"Fetching {series_name} data...")
        series_data = fred.get_series(series_id, start_date, end_date)
        all_data[series_name] = series_data
    
    # Combine all series into a single dataframe
    df = pd.DataFrame(all_data)
    
    # Calculate 2Y-10Y spread and inversion flag
    if '2_year_yield' in df.columns and '10_year_yield' in df.columns:
        df['spread_2y_10y'] = df['10_year_yield'] - df['2_year_yield']
        df['yield_curve_inverted'] = (df['spread_2y_10y'] < 0).astype(int)
    
    # Forward fill missing values for better time series continuity
    df = df.ffill()
    
    print(f"Data fetched successfully. Shape: {df.shape}")
    return df

# 2. Enhanced Feature Engineering
def create_features(df, lags=[1, 5, 20]):
    """Generate lagged yields, term spreads, and rolling metrics for all indicators."""
    df_features = df.copy()
    
    # Yield and indicator columns
    yield_cols = ['2_year_yield', '3_year_yield', '5_year_yield', '7_year_yield', '10_year_yield', '20_year_yield']
    macro_cols = ['fed_funds_rate', 'core_cpi', 'fed_assets', 'usd_index']
    
    # All columns to process
    all_cols = yield_cols + macro_cols + ['spread_2y_10y']
    
    # 1. Create lagged features for all columns
    for col in all_cols:
        if col in df_features.columns:  # Check if column exists
            for lag in lags:
                df_features[f'{col}_lag{lag}'] = df_features[col].shift(lag)
    
    # 2. Create change features (1-day, 5-day, 20-day changes)
    for col in all_cols:
        if col in df_features.columns:
            df_features[f'{col}_1d_chg'] = df_features[col].diff(1)
            df_features[f'{col}_5d_chg'] = df_features[col].diff(5)
            df_features[f'{col}_20d_chg'] = df_features[col].diff(20)
    
    # 3. Create rolling metrics (5-day and 30-day) for all columns
    for col in all_cols:
        if col in df_features.columns:
            df_features[f'{col}_ma5'] = df_features[col].rolling(window=5).mean()
            df_features[f'{col}_ma30'] = df_features[col].rolling(window=30).mean()
            df_features[f'{col}_vol5'] = df_features[col].rolling(window=5).std()
            df_features[f'{col}_vol30'] = df_features[col].rolling(window=30).std()
    
    # 4. Calculate Fed Assets YoY change
    if 'fed_assets' in df_features.columns:
        df_features['fed_assets_yoy'] = df_features['fed_assets'].pct_change(252) * 100
    
    # 5. Maintain original term spreads
    df_features['spread_10y_3y'] = df_features['10_year_yield'] - df_features['3_year_yield']
    df_features['spread_20y_10y'] = df_features['20_year_yield'] - df_features['10_year_yield']
    df_features['spread_20y_3y'] = df_features['20_year_yield'] - df_features['3_year_yield']
    
    # Drop rows with NaN values due to lagging/rolling
    df_features = df_features.dropna()
    return df_features

# 3. Prepare Data for Modeling with Train/Test Split
def prepare_data(df_features, target_col='10_year_yield', test_size=0.2):
    """Split features and target, create train/test sets using an 80/20 time-series split."""
    # Target: Next day's 10-year yield
    df_features['target'] = df_features[target_col].shift(-1)
    df_features = df_features.dropna()
    
    # Features: All columns except target and original macro inputs
    exclude_cols = ['target', 'cpi', 'vix', 'gdp']
    feature_cols = [col for col in df_features.columns if col not in exclude_cols]
    
    # Create 80/20 time-series split
    split_idx = int(len(df_features) * (1 - test_size))
    
    # Train set
    X_train = df_features[feature_cols].iloc[:split_idx]
    y_train = df_features['target'].iloc[:split_idx]
    
    # Test set
    X_test = df_features[feature_cols].iloc[split_idx:]
    y_test = df_features['target'].iloc[split_idx:]
    
    print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")
    
    return X_train, X_test, y_train, y_test, feature_cols

# 4. Train Voting Ensemble Model (XGBoost + RandomForest)
def train_ensemble_model(X_train, y_train, param_grid=None):
    """Train a VotingRegressor with XGBoost and RandomForest models."""
    # XGBoost model
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    
    # RandomForest model
    rf_model = RandomForestRegressor(random_state=42)
    
    # Default parameter grid for XGBoost if none provided
    if param_grid is None:
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1.0]
        }
    
    # Time-series cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    
    # Grid search for XGBoost hyperparameter tuning
    grid_search = GridSearchCV(
        xgb_model, param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1
    )
    grid_search.fit(X_train, y_train)
    
    # Best XGBoost model
    best_xgb_model = grid_search.best_estimator_
    print(f"Best XGBoost parameters: {grid_search.best_params_}")
    
    # Train RandomForest with default parameters
    rf_model.fit(X_train, y_train)
    
    # Create and train VotingRegressor with weights 0.8/0.2
    ensemble = VotingRegressor(
        estimators=[('xgb', best_xgb_model), ('rf', rf_model)],
        weights=[0.8, 0.2]
    )
    ensemble.fit(X_train, y_train)
    
    return ensemble, grid_search

# 5. Model Evaluation on Test Set
def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Evaluate model performance with RMSE, MAE, and directional accuracy on train and test sets."""
    # Train set predictions
    train_preds = model.predict(X_train)
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    train_mae = mean_absolute_error(y_train, train_preds)
    
    # Directional accuracy - training
    actual_diff_train = np.sign(y_train - y_train.shift(1).fillna(0))
    pred_diff_train = np.sign(pd.Series(train_preds, index=y_train.index) - y_train.shift(1).fillna(0))
    train_dir_acc = np.mean(actual_diff_train == pred_diff_train) * 100
    
    # Test set predictions
    test_preds = model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
    test_mae = mean_absolute_error(y_test, test_preds)
    
    # Directional accuracy - test
    actual_diff_test = np.sign(y_test - y_test.shift(1).fillna(0))
    pred_diff_test = np.sign(pd.Series(test_preds, index=y_test.index) - y_test.shift(1).fillna(0))
    test_dir_acc = np.mean(actual_diff_test == pred_diff_test) * 100
    
    # Print metrics
    print("\n--- Training Set Performance ---")
    print(f"RMSE: {train_rmse:.4f}")
    print(f"MAE: {train_mae:.4f}")
    print(f"Directional Accuracy: {train_dir_acc:.2f}%")
    
    print("\n--- Test Set Performance ---")
    print(f"RMSE: {test_rmse:.4f}")
    print(f"MAE: {test_mae:.4f}")
    print(f"Directional Accuracy: {test_dir_acc:.2f}%")
    
    # Plot predictions vs actual for test set
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual 10Y Yield', color='blue')
    plt.plot(y_test.index, test_preds, label='Predicted 10Y Yield', color='red', linestyle='--')
    plt.title('Test Set: Actual vs Predicted 10-Year Treasury Yield')
    plt.xlabel('Date')
    plt.ylabel('Yield (%)')
    plt.legend()
    plt.grid(True)
    plt.savefig('yield_predictions.png')
    plt.close()
    
    # Feature importance plot (using XGBoost model)
    xgb_model = model.named_estimators_['xgb']
    feature_importance = xgb_model.get_booster().get_score(importance_type='gain')
    sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:15]
    plt.figure(figsize=(12, 8))
    plt.barh([x[0] for x in sorted_importance], [x[1] for x in sorted_importance])
    plt.title('Top 15 Feature Importance')
    plt.xlabel('Gain')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()
    
    return {
        'train_rmse': train_rmse, 
        'train_mae': train_mae, 
        'train_dir_acc': train_dir_acc,
        'test_rmse': test_rmse, 
        'test_mae': test_mae, 
        'test_dir_acc': test_dir_acc
    }

# Main Execution
if __name__ == "__main__":
    # Fetch enhanced data using FRED API
    # You can either set FRED_API_KEY as an environment variable or pass it directly
    # Example: df = fetch_fred_data(api_key='your_api_key_here')
    df = fetch_fred_data(api_key='cc07b760ce5e5ddcd406da43e91f6889')
    
    # Create enhanced features
    df_features = create_features(df)
    
    # Prepare data with 80/20 time-series split
    X_train, X_test, y_train, y_test, feature_cols = prepare_data(df_features)
    
    # Train ensemble model
    ensemble_model, grid_search = train_ensemble_model(X_train, y_train)
    
    # Evaluate model on train and test sets
    metrics = evaluate_model(ensemble_model, X_train, y_train, X_test, y_test)
    
    # Generate next-day forecast (using the existing function)
    latest_date = df_features.index[-1]
    X_latest = df_features.loc[[latest_date]][feature_cols].values
    forecast = ensemble_model.predict(X_latest)[0]
    print(f"\nForecasted 10Y Yield for {latest_date + timedelta(days=1)}: {forecast:.4f}%")
    
    # Save model and performance report
    import joblib
    joblib.dump(ensemble_model, 'treasury_yield_ensemble_model.pkl')
    
    # Save performance report
    with open('performance_report.txt', 'w') as f:
        f.write("10-Year Treasury Yield Forecast Performance Report\n")
        f.write(f"Date: {datetime.today().strftime('%Y-%m-%d')}\n\n")
        f.write(f"Best XGBoost Parameters: {grid_search.best_params_}\n\n")
        f.write("--- Training Set Performance ---\n")
        f.write(f"RMSE: {metrics['train_rmse']:.4f}\n")
        f.write(f"MAE: {metrics['train_mae']:.4f}\n")
        f.write(f"Directional Accuracy: {metrics['train_dir_acc']:.2f}%\n\n")
        f.write("--- Test Set Performance ---\n")
        f.write(f"RMSE: {metrics['test_rmse']:.4f}\n")
        f.write(f"MAE: {metrics['test_mae']:.4f}\n")
        f.write(f"Directional Accuracy: {metrics['test_dir_acc']:.2f}%\n\n")
        f.write(f"Forecast for {latest_date + timedelta(days=1)}: {forecast:.4f}%\n")
        f.write("\nPlots saved: yield_predictions.png, feature_importance.png")
        f.write("\nModel saved: treasury_yield_ensemble_model.pkl")

Fetching 3_year_yield data...
Fetching 5_year_yield data...
Fetching 7_year_yield data...
Fetching 10_year_yield data...
Fetching 20_year_yield data...
Fetching cpi data...
Fetching fed_funds_rate data...
Fetching vix data...
Fetching 2_year_yield data...
Fetching core_cpi data...
Fetching fed_assets data...
Fetching usd_index data...
Fetching gdp data...
Data fetched successfully. Shape: (6706, 15)
Training data shape: (4071, 126), Test data shape: (1018, 126)
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best XGBoost parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}

--- Training Set Performance ---
RMSE: 0.0413
MAE: 0.0313
Directional Accuracy: 64.95%

--- Test Set Performance ---
RMSE: 0.1208
MAE: 0.0956
Directional Accuracy: 48.33%

Forecasted 10Y Yield for 2025-05-15 00:00:00: 4.4342%
